# Entorno Remoto de Compilación (MambaVision) 🚀

Este notebook se ejecuta enteramente en los servidores de Google Colab conectado desde tu VS Code.
Sigue los pasos secuenciales para descargar tu código desde GitHub y compilar el Kernel Custom de C++ contra los headers de TensorFlow provistos por Google.

In [ ]:
import os
import sys

# ==========================================
# CONFIGURACIÓN DE TU REPOSITORIO DE GITHUB
# ==========================================
GITHUB_REPO = "https://github.com/rdromer2/mambavision_tf_kernel.git"
REPO_NAME = "mambavision_tf_kernel"
PROJECT_PATH = f"/content/{REPO_NAME}"

# 1. Clonar o actualizar el repositorio
if not os.path.exists(PROJECT_PATH):
    print(f"Clonando el repositorio {GITHUB_REPO}...")
    os.system(f'git clone {GITHUB_REPO} {PROJECT_PATH}')
else:
    print("El repositorio ya existe. Actualizando con git pull...")
    os.system(f'cd {PROJECT_PATH} && git pull')

# 2. Añadir al sys.path
sys.path.append(PROJECT_PATH)

# 3. Cambiar nuestro directorio activo al proyecto
os.chdir(PROJECT_PATH)
print(f"\nDirectorio de trabajo actual: {os.getcwd()}")

## 🛠️ Compilación C++ con g++ directo
Compilamos usando los flags exactos que reporta `tf.sysconfig`, sin intermediarios (CMake/Make).

In [ ]:
import tensorflow as tf

TF_CFLAGS = ' '.join(tf.sysconfig.get_compile_flags())
TF_LFLAGS = ' '.join(tf.sysconfig.get_link_flags())

print('TF compile flags:', TF_CFLAGS)
print('TF link flags:', TF_LFLAGS)
print()

os.chdir(PROJECT_PATH)
compile_cmd = f"g++ -std=c++17 -shared zero_out.cc -o zero_out.so -fPIC {TF_CFLAGS} {TF_LFLAGS} -O2"
print('Ejecutando:', compile_cmd)
result = os.system(compile_cmd)
if result == 0:
    print('\n✅ Compilación exitosa: zero_out.so creado.')
else:
    print(f'\n❌ Error en la compilación. Código: {result}')

## 🧪 Prueba Inicial del Kernel
**Clave**: Antes de cargar nuestro `.so`, forzamos a que los símbolos de `libtensorflow_framework` sean visibles globalmente usando `ctypes.RTLD_GLOBAL`. Sin este paso, `dlopen` no puede resolver los símbolos de TF dentro de nuestra librería.

In [ ]:
import ctypes
import tensorflow as tf
import glob

# === EL FIX CLAVE ===
# Cargar libtensorflow_framework.so con RTLD_GLOBAL para que sus símbolos
# estén disponibles cuando dlopen cargue nuestro zero_out.so
tf_lib_dir = tf.sysconfig.get_lib()
fw_libs = glob.glob(os.path.join(tf_lib_dir, 'libtensorflow_framework.so*'))
if fw_libs:
    # Cargar la primera que encontremos (suele ser .so.2)
    lib_path = sorted(fw_libs)[-1]
    print(f'Precargando: {lib_path}')
    ctypes.CDLL(lib_path, ctypes.RTLD_GLOBAL)
else:
    print('⚠️ No se encontró libtensorflow_framework.so')

# Ahora sí, cargar nuestra operación custom
try:
    zero_out_module = tf.load_op_library(f'{PROJECT_PATH}/zero_out.so')
    result = zero_out_module.zero_out([[1, 2], [3, 4]])
    print(f'\n🎉 ¡ÉXITO! Resultado: {result}')
except Exception as e:
    print('Error al cargar/ejecutar la librería:', e)